In [9]:
%load_ext autoreload
%autoreload 2
from yahpo_train.cont_normalization import ContNormalization
from yahpo_train.model  import *
from yahpo_train.metrics import *
from yahpo_gym.benchmarks import lcbench, rbv2
from yahpo_gym.configuration import cfg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
cc = cfg('rbv2_glmnet')
dls = dl_from_config(cc, bs=2048)

In [10]:
f = FFSurrogateModel(dls, layers=[512,512])
l = SurrogateTabularLearner(dls, f, loss_func=nn.MSELoss(reduction='mean'), metrics=nn.MSELoss)
l.metrics = [AvgTfedMetric(mae),  AvgTfedMetric(r2), AvgTfedMetric(spearman)]
l.add_cb(MixHandler)
l.add_cb(EarlyStoppingCallback(patience=3))

NLL YJ: -2882590.9946846357
NLL Scaler: -3832622.3806794905
NLL Scaler: nan
NLL Scaler: nan
NLL Scaler: -4061630.370804108
NLL YJ: 6169796.717844549
NLL Scaler: 2499197.895726821
NLL Scaler: 1793369.409136417
NLL Scaler: 4898619.587748892
NLL Scaler: -2082691.5082045258
NLL YJ: -2888510.3327216604
NLL Scaler: -3836567.0950524095
NLL Scaler: -2543764.725332674
NLL Scaler: 561485.2380746134
NLL Scaler: -4136480.030878919
NLL YJ: 7373113.906852744
NLL Scaler: 465265.1473603297
NLL Scaler: -1396655.7581372545
NLL Scaler: 1708594.5248908843
NLL Scaler: -6253908.921465752
NLL YJ: -3886496.2129677013
NLL Scaler: -4435837.95078848
NLL Scaler: nan
NLL Scaler: nan
NLL Scaler: -4602672.549334051
NLL YJ: -2169029.680202037
NLL Scaler: -3031860.049233366
NLL Scaler: nan
NLL Scaler: nan
NLL Scaler: -3231503.4697811585
NLL YJ: -5434002.887308268
NLL Scaler: -5847599.219139412
NLL Scaler: -5664787.009221261
NLL Scaler: -2559537.2431321847
NLL Scaler: -6293085.394365977
NLL YJ: -1914924.2507311567
NLL 

In [13]:
[x.scaler for x in l.embds_dbl] + [x.scaler for x in l.embds_tgt]

 None]

In [20]:
l.fit_flat_cos(5, 1e-3)
for p in l.model.wide.parameters():
    p.requires_grad = False
l.fit_flat_cos(5, 1e-4)

epoch,train_loss,valid_loss,mae,r2,spearman,time
0,0.005727,0.004985,[0.03071178 0.03134002 0.22002012 0.0893494 5.2209722 0.08186299],[ 0.91939017 0.96216957 -0.02699783 0.84406365 0.02074665 0.59069368],[0.92895188 0.89435466 0.70853261 0.91897454 0.71040431 0.90916701],03:50
1,0.004924,0.004805,[0.02959018 0.03119148 0.21869054 0.09277981 5.211237 0.07693558],[ 0.91571131 0.96586696 -0.05582809 0.80882739 0.01837675 0.62739999],[0.93200834 0.89901171 0.485341 0.90717091 0.68058017 0.90884171],03:51
2,0.004607,0.004586,[0.02676723 0.02445112 0.21325743 0.08717156 5.22018653 0.0751325 ],[ 0.9180749 0.97025915 -0.01602825 0.80046624 0.02038021 0.63474804],[0.93628577 0.90634329 0.60087258 0.90953818 0.66442099 0.90373651],03:51
3,0.004433,0.004878,[0.02837784 0.02633818 0.21872403 0.09404772 5.19595637 0.07525598],[ 0.92199831 0.96338459 -0.06185373 0.78239802 0.02522013 0.62443524],[0.9357698 0.90369516 0.40442538 0.89982081 0.67237838 0.89769475],03:52
4,0.004222,0.004666,[0.0251466 0.02458286 0.2133736 0.08416838 5.2053894 0.0749461 ],[ 0.92358888 0.96644786 -0.03229268 0.78446398 0.02389078 0.62361928],[0.93503215 0.9047265 0.5045005 0.8977712 0.6712361 0.89269219],03:52


epoch,train_loss,valid_loss,mae,r2,spearman,time
0,0.004216,0.004673,[0.02524103 0.0245316 0.21280765 0.08484684 5.21068137 0.07540551],[ 0.92661092 0.96637061 -0.02811578 0.78393324 0.02250905 0.61715866],[0.93458675 0.90504417 0.52882816 0.89878155 0.67181911 0.88967823],03:51
1,0.004159,0.004769,[0.02574775 0.02426513 0.21297741 0.08609769 5.20385468 0.07493126],[ 0.92098981 0.96533756 -0.02518601 0.77481419 0.02321787 0.61824871],[0.9340293 0.90441931 0.48954833 0.89393443 0.67454495 0.88872414],03:51
2,0.004176,0.004732,[0.02525586 0.02410149 0.21396262 0.08792935 5.19457071 0.07562068],[ 0.92380711 0.96659612 -0.03715277 0.7663898 0.02456437 0.61749188],[0.93574685 0.9049795 0.45129179 0.89307266 0.69069685 0.88344027],03:46
3,0.004184,0.004750,[0.02544605 0.02528027 0.21303462 0.08676766 5.18418809 0.07531716],[ 0.92253676 0.96622385 -0.03302443 0.76761085 0.02903252 0.61915603],[0.93463414 0.90369132 0.487074 0.8928585 0.68202158 0.88543753],03:45


No improvement since epoch 0: early stopping


In [ ]:
l.export_onnx(cfg)

c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\cont_normalization.py:50: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.abs(lmbda) < self.eps:
c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\cont_normalization.py:56: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.abs(lmbda - 2.) <= self.eps:
c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\model.py:138: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value wil

In [56]:
 class Scaler():
    def  __init__(self, forward, invert):
        self.fwd = forward
        self.inv = invert

    def forward(self, x):
            return self.fwd(x)

    def invert(self,x):
        return self.inv(x)

 scs = [Scaler(torch.log1p, torch.expm1), Scaler(torch.log10, lambda x: torch.pow(10., x)),Scaler(torch.log2, torch.exp2), Scaler(lambda x: torch.exp(-x), lambda x: - torch.log(x))]

In [57]:
x= torch.rand(100)
for sc in scs:
    fwd = sc.forward(x)
    print(torch.mean(sc.invert(fwd) - x))

tensor(-4.3213e-09)
tensor(-7.3342e-10)
tensor(-1.9278e-09)
tensor(-3.3295e-10)


tensor([ 0.0000e+00,  0.0000e+00,  5.9605e-08,  0.0000e+00,  0.0000e+00,
        -5.9605e-08,  0.0000e+00,  0.0000e+00, -5.9605e-08,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  5.9605e-08, -5.9605e-08,  0.0000e+00,
        -5.9605e-08,  0.0000e+00,  5.9605e-08,  5.9605e-08,  5.9605e-08,
         5.9605e-08,  0.0000e+00,  5.9605e-08, -5.9605e-08,  5.9605e-08,
        -5.9605e-08,  0.0000e+00,  5.9605e-08,  5.9605e-08, -5.9605e-08,
         0.0000e+00,  5.9605e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  5.9605e-08,  0.0000e+00,  5.9605e-08,  5.9605e-08,
         5.9605e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         5.9605e-08,  0.0000e+00, -5.9605e-08, -5.9605e-08,  0.0000e+00,
        -5.9605e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00, -5.9605e-08,  0.0000e+00, -5.9605e-08, -5.9605e-08,
         0.0000e+00,  0.0000e+00, -5.9605e-08,  0.0000e+00,  5.9605e-08,
         0.0000e+00,  0.0000e+00,  5.9605e-08, -5.9